In [11]:
from itertools import count
import json
from pathlib import Path
import numpy as np
import IPython.display as disp
from osgeo import gdal
import pandas as pd

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWi4pcCgHmcwO7lfvPAO8WJ2rv7YsfQubCZK4MswHmbsCYVMsS5DpJs

Successfully saved authorization token.


In [2]:
dataset = ee.ImageCollection('MODIS/006/MOD13Q1').filter(ee.Filter.date('2020-01-01', '2020-12-31'))

In [3]:
ndvi = dataset.select('NDVI')

In [4]:
print(ndvi)

ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.map",
    "arguments": {
      "baseAlgorithm": {
        "functionDefinitionValue": {
          "argumentNames": [
            "_MAPPING_VAR_0_0"
          ],
          "body": {
            "functionInvocationValue": {
              "functionName": "Image.select",
              "arguments": {
                "bandSelectors": {
                  "constantValue": [
                    "NDVI"
                  ]
                },
                "input": {
                  "argumentReference": "_MAPPING_VAR_0_0"
                }
              }
            }
          }
        }
      },
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "ImageCollection.load",
                "arguments": {
                  "id": {
   

In [5]:
import folium
# Define the center of our map.
lat, lon = 23.730104, 90.306526
my_map = folium.Map(location=[lat, lon], zoom_start=10)
my_map

In [6]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [7]:
meanNDVI = ndvi.mean().multiply(0.0001)
vis_params = {
  'min': 0.0,
  'max': 1.0,
  'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901','66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01','012E01', '011D01', '011301'],
} 
# Define the center of our map.
lat, lon = 23.730104, 90.306526

my_map = folium.Map(location=[lat, lon], zoom_start=8)

my_map.add_ee_layer(meanNDVI, vis_params, 'NDVI')

In [8]:
# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

# calculating mean value of NDVI

In [18]:
geojson_path1 = list(Path(Path.cwd()/"Data"/"GeoJson").resolve().iterdir())[0]

countries = ['Malawi']
# get coordinates of the country
with open(geojson_path1) as f:
    data = json.load(f)
    coords = {}
    for j in countries:
        for i in data['features']:
            if i['properties']['name'] == j:
                coords[j] = i['geometry']['coordinates']
                break

In [19]:
#Define a region of interest as a point and buffer 1km around the area
# roi = ee.Geometry.Point([-124.0769, 40.1035]).buffer(1000)
roi = ee.Geometry.Polygon(coords['Malawi'])

#Get Image from Landsat

image = ee.Image(ee.ImageCollection('LANDSAT/LC08/C01/T1_RT')\
                     .filterDate('2014-01-01', '2014-01-01')\
                     .filterBounds(roi))

#Function from https://gis.stackexchange.com/q/360278

def meanNDVICollection (img,aoi):
    nir = img.select('B5')
    red = img.select('B4')
    ndviImage = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

    # Compute the mean of NDVI over the 'region'
    ndviValue = ndviImage.reduceRegion(**{
    'geometry': aoi.getInfo(),
    'reducer': ee.Reducer.mean(),
    }).get('NDVI');  # result of reduceRegion is always a dictionary, so get the element we want

    newFeature = ee.Feature(None, {
      # Adding computed NDVI value
      'NDVI': ndviValue
    }).copyProperties(img, [
      # Picking properties from original image
      'system:time_start'
    ])

    return newFeature

In [20]:
x = meanNDVICollection(image,roi)

In [21]:
# print(x.get('NDVI').getInfo())

In [22]:
print(x)

ee.Element({
  "functionInvocationValue": {
    "functionName": "Element.copyProperties",
    "arguments": {
      "destination": {
        "functionInvocationValue": {
          "functionName": "Feature",
          "arguments": {
            "metadata": {
              "dictionaryValue": {
                "values": {
                  "NDVI": {
                    "functionInvocationValue": {
                      "functionName": "Dictionary.get",
                      "arguments": {
                        "dictionary": {
                          "functionInvocationValue": {
                            "functionName": "Image.reduceRegion",
                            "arguments": {
                              "geometry": {
                                "functionInvocationValue": {
                                  "functionName": "GeometryConstructors.Polygon",
                                  "arguments": {
                                    "coordinates": {
                 

In [23]:
# calculate monthly average NDVI and plot against yield?